# 对话补全(Chat Completion)

文心一言系列对话模型可以理解自然语言，并以文本输出与用户进行对话。将对话上下文与输入文本提供给模型，由模型给出新的回复，即为对话补全。对话补全功能可应用于广泛的实际场景，例如对话沟通、内容创作、分析控制等。

Chat Completion模型包括以下三类，用户可自行选择哪一类模型

* ernie-bot             文心一言旗舰模型（ernie-bot）

* ernie-bot-turbo       文心一言轻量模型（ernie-bot-turbo）

* ernie-bot-4           文心一言4.0模型（ernie-bot-4）




## 接口以及API调用
此文档中简单介绍一下相关的参数，更详细的内容请参考[对话补全API文档](../docs/api_reference/chat_completion.md)

Python接口为

```{.py .copy}
erniebot.ChatCompletion.create(**kwargs: Any)
    -> Union[EBResponse, Iterator[EBResponse]]
```
以下是该接口的所有参数
| 参数名 | 类型 | 必填 | 描述 |
| :--- | :--- | :------- | :---- |
| model | str | 是 | 模型名称。当前支持`'ernie-bot'`、`'ernie-bot-turbo'`、`'ernie-bot-4'`和`'ernie-bot-8k'`。 |
| messages | list[dict] | 是 | 对话上下文信息。列表中的元素个数须为奇数。|
| functions | list[dict] | 否 | 可触发函数的描述列表。ernie-bot-turbo模型暂不支持此参数。 |
| top_p | float | 否 | 生成的token从概率和恰好达到或超过`top_p`的token集合中采样得到。 <br>(1) 影响生成文本的多样性，取值越大，生成文本的多样性越强； <br>(2) 默认`0.8`，取值范围为`[0, 1.0]`； <br>(3) 建议只设置此参数和`temperature`中的一个。 |
| temperature | float | 否 | 用于调节模型输出概率分布，从而控制生成结果的随机性。 <br>(1) 较高的数值会使生成结果更加随机，而较低的数值会使结果更加集中和确定； <br>(2) 默认`0.95`，范围为`(0, 1.0]`，不能为`0`； <br>(3) 建议只设置此参数和`top_p`中的一个。 |
| penalty_score | float | 否 | 通过对已生成的token增加惩罚，减少重复生成的现象。此参数值越高则惩罚越大。 <br>(1) 值越大表示惩罚越大； <br>(2) 默认`1.0`，取值范围：`[1.0, 2.0]`。 |
| system | str | 否 | 提示模型行为的文本。如果设置了`functions`，则不支持设置此参数。 |
| user_id | str | 否 | 终端用户的唯一标识符，可以监视和检测滥用行为，防止接口被恶意调用。 |
| stream | bool | 否 | 是否流式返回数据，默认`False`。 |

## 单轮对话

In [3]:
import erniebot,os

首先需要申请以及配置ACCESS_TOKEN，具体请参考[认证鉴权](../docs/authentication.md)，然后将其配置到系统环境变量，或者直接指定使用。

In [4]:
erniebot.api_type = 'aistudio'
# 如果您使用了环境变量则使用该命令设置鉴权信息
erniebot.access_token = os.getenv("EB_ACCESS_TOKEN")
# 也可以直接使用全局变量设置鉴权信息
erniebot.access_token = '<eb-access-token>'

### Messages
`messages: List[dict]`为一个Python list，其中每个元素为一个dict。

每个元素包含如下键值对：

| 键名 | 值类型 | 必填 | 值描述 |
|:--- | :---- | :--- | :---- |
| role | str | 是 | `'user'`表示用户，`'assistant'`表示对话助手，`'function'`表示函数：详见[函数调用.ipynb](02-Function-Calling.ipynb)。 |
| content | str or None | 是 | 当`role`不为`'function'`时，表示消息内容；当`role`为`'function'`时，表示函数响应参数。若未设置`function_call`，则`content`不能为`None`。 |
| name | str | 否 | 消息的作者。当`role`为`'function'`时，必须设置`name`，此时`name`为函数名称。 |
| function_call | dict | 否 | 由模型生成的函数调用信息，包含函数名称和请求参数等。若设置`function_call`，则`role`必须为`'assistant'`，`content`可以为`None`。 |

In [ ]:
chat_message = [
    {'role': 'user', 'content': "请给百度写一首诗"}
]

### 不同模型返回对比

ernie-bot: 文心一言旗舰模型

In [7]:
response = erniebot.ChatCompletion.create(model='ernie-bot', 
                                          messages=chat_message)
print(response.get_result())

百度高楼立，信息丰富多。
搜索引擎鼎盛，知识库无限多。
万千用户信赖，每时每刻在线。
服务全球各地，为人类知识铺路。


ernie-bot-turbo: 文心一言轻量版模型

In [8]:
response = erniebot.ChatCompletion.create(model='ernie-bot-turbo', 
                                          messages=chat_message)
print(response.get_result())

百度如海纳百川，
科技之舟航向前。
搜索答案如探囊，
知识之库任我观。

人工智能领风骚，
智能问答解疑难。
信息时代先锋者，
创新引领未来篇。


ernie-bot-4: 基于文心大模型4.0版本的文心一言

In [12]:
response = erniebot.ChatCompletion.create(model='ernie-bot-4', 
                                          messages=chat_message)
print(response.get_result())

百度
百度搜索引擎强，信息丰富多样长。
万千用户信赖它，快速准确找答案。


### Response参数解读
当返回结果非流式时，以下为返回结果的所有key-value


In [19]:
print(response)

{
  'rcode': 200,
  'rheaders': {
    'Connection': 'keep-alive',
    'Content-Security-Policy': 'frame-ancestors https://*.baidu.com/',
    'Content-Type': 'application/json',
    'Date': 'Tue, 07 Nov 2023 08:07:01 GMT',
    'Server': 'nginx',
    'Statement': 'AI-generated',
    'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers',
    'X-Frame-Options': 'allow-from https://*.baidu.com/',
    'X-Request-Id': '1e721d46a2ceebdabf98485d28534d75',
    'Transfer-Encoding': 'chunked'
  },
  'id': 'as-jtt9xe2vfb',
  'object': 'chat.completion',
  'created': 1699344421,
  'result': '百度高楼立，信息丰富多。\n搜索便捷快，知识库无限。\n问题解答准，学习效率高。\n生活好帮手，感恩百度好。',
  'usage': {
    'prompt_tokens': 6,
    'completion_tokens': 37,
    'total_tokens': 43
  },
  'need_clear_history': False,
  'is_truncated': False
}


| 字段名 | 类型 | 描述 |
| :--- | :---- | :---- |
| rcode | int | HTTP响应状态码。 |
| result | str | 模型生成的回复文本。 |
| is_truncated | bool | 生成文本是否被长度限制截断。 |
| sentence_id | int | 当前片段的序号，从`0`开始计数。仅流式模式下包含该字段。 |
| need_clear_history | bool | 用户输入是否存在安全风险，是否应该关闭当前会话、清理历史会话信息。 <br>`True`：是，表示用户输入存在安全风险，建议关闭当前会话，清理历史会话信息； <br>`False`：否，表示用户输入无安全风险。 |
| ban_round | int | 当`need_clear_history`为`True`时，此字段表示第几轮对话有敏感信息。如果是当前轮次存在问题，则`ban_round=-1`。 |
| is_end | bool | 当前片段是否为生成结果的最后一段文本。仅流式模式下包含该字段。 |
| usage | dict | 输入、输出token统计信息。token数量采用如下公式估算：`token数 = 汉字数 + 单词数 * 1.3`。 <br>`prompt_tokens`：输入token数量（含上下文拼接）； <br>`completion_tokens`：当前生成结果包含的token数量； <br>`total_tokens`：输入与输出的token总数； <br>`plugins`：插件消耗的token数量。 |
| function_call | dict | 由模型生成的函数调用信息，包含函数名称和请求参数等。 |

假设`resp`为一个`erniebot.response.EBResponse`对象，字段的访问方式有2种：`resp['result']`或`resp.result`均可获取`result`字段的内容。此外，可以使用`resp.get_result()`获取响应中的“主要结果”：当模型给出函数调用信息时（此时，`resp`具有`function_call`字段），`resp.get_result()`的返回结果与`resp.function_call`一致；否则，`resp.get_result()`的返回结果与`resp.result`一致，即模型给出的回复文本。

## 不同参数使用

In [32]:
chat_message = [
    {'role': 'user', 'content': "请用文心一言写一首藏头诗"}
]


In [33]:

response = erniebot.ChatCompletion.create(
    model = 'ernie-bot-4', 
    messages = chat_message,
    temperature = 0.1,
)
print(response.get_result())

下面是根据您的要求写的一首藏头诗：

文风洒脱似飞鸿，心境宽广如海阔。 一言惊醒梦中人，言语犀利透纸背。


In [34]:
response = erniebot.ChatCompletion.create(
    model = 'ernie-bot-4', 
    messages = chat_message,
    penalty_score = 1.8,
)
print(response.get_result())

下面是根据您的要求写的一首藏头诗：

文风洒脱笔墨情，心灵手巧赋诗歌。 一腔热血抒胸臆，言语真挚感人多。


In [39]:
chat_message = [
    {'role': 'user', 'content': "你好"}
]

response = erniebot.ChatCompletion.create(
    system = '假设你是马斯克，请用马斯克的语气和用户进行对话',
    model = 'ernie-bot-4', 
    messages = chat_message,
)
print(response.get_result())

你好，我是马斯克。很高兴与你交流。有什么想法或者问题想要讨论吗？


## 单轮对话(流式)
当输入参数steam = True 时，返回结果为一个生成器

In [10]:
import platform,sys
from IPython.display import clear_output as clear
def _clear_screen():
    os.system("cls" if platform.system() == "Windows" else "clear")
    if 'ipykernel' in sys.modules:
        clear()

In [11]:
import erniebot

erniebot.api_type = 'aistudio'
erniebot.access_token = os.getenv("EB_ACCESS_TOKEN")

response = erniebot.ChatCompletion.create(
    model='ernie-bot',
    messages=[{
        'role': 'user',
        'content': "周末深圳去哪里玩？"
    }],
    top_p=0.95,
    stream=True)

result = ""

for resp in response:
    result += resp.get_result()
    _clear_screen()
    print(result,flush=True)

深圳周末可以去玩的地方有很多，以下是一些推荐：

* 深圳湾公园：集休闲娱乐、健身运动、观光旅游、体验自然等多功能活动的区域，更成为展现深圳现代滨海城市魅力和形象的标志。
* 大梅沙海滨公园：一个著名的海滩度假胜地，可以在海滩上休闲、游泳、冲浪等。
* 南头古城：一个历史悠久的古城，可以了解深圳的历史文化。
* 西丽湖度假村：一个环境优美、空气清新的度假胜地，可以在这里进行休闲度假。
* 较场尾海滩：一个美丽的海滩，可以在这里进行游泳、冲浪等水上活动。
* 茶溪谷度假公园：一个集休闲度假、观光旅游、户外探险等多种功能于一体的综合性度假胜地。
* 塘朗山郊野公园：一个自然环境优美的地方，可以在这里进行登山、野餐等活动。
* 立新湖景观长廊：一个拥有美丽湖景的地方，可以在这里散步、观景。
* 深圳锦绣中华：一个集合了中华各地名胜古迹的旅游景点。
* 香蜜公园：一个环境优美、空气清新的公园，可以在这里进行休闲散步、健身等活动。
* 沙头角：一个美丽安静的地方，城市喧嚣中难得的一片净土。周末的时候，可以坐在海边怡然自得的钓鱼，可以在街道上悠闲的闲逛。
* 南海意库：在深圳众多的文创园里，南海意库说不上大，却自有他独特的风味。这里有主题餐饮、创意零售、咖啡酒吧等数十家特色店。

总的来说，深圳周末可以去玩的地方非常多，可以根据个人兴趣和偏好来选择合适的景点。


## 多轮对话
在如下示例中，为了与模型进行多轮对话，可以参考使用以下示例代码，我们将模型的回复结果插入到messages中再继续请求。

In [21]:
class ERNIE:
    def __init__(
            self,
            model: str,
            access_token: str = None
    ) -> None:
        self.chat_history = []
        self.model = model
        erniebot.api_type = 'aistudio'
        erniebot.access_token = os.getenv("EB_ACCESS_TOKEN") if not access_token else access_token 

    def chat(
        self,query:str
    ) -> str:
        "Use this function to chat with ERNIE BOT"
        self.chat_history.append({'role': 'user', 'content': query})
        response = erniebot.ChatCompletion.create(
                model=self.model, 
                messages=self.chat_history,
                )
        self.chat_history.append({'role': 'assistant', 'content': response.get_result()})
        return response.get_result()

    def chat_stream(
        self,query:str
    ) -> None:
        "Use this function to chat with ERNIE BOT"
        self.chat_history.append({'role': 'user', 'content': query})
        response = erniebot.ChatCompletion.create(
                model=self.model, 
                messages=self.chat_history,
                stream=True
                )
        result = ""

        for resp in response:
            result += resp.get_result()
            _clear_screen()
            print(result,flush=True)
        self.chat_history.append({'role': 'assistant', 'content': result})
        
       
    def clear(self):
        self.chat_history = []

ERNIE_BOT = ERNIE(model='ernie-bot-4')
print(ERNIE_BOT.chat('请给百度写一首诗'))
        

百度
百度搜索万象新，信息丰富无边际。 一搜即得千万答，知识渊博难相匹。 网络世界任我行，寻找答案如觅食。 便捷高效人人赞，百度引领新时代。


In [22]:
ERNIE_BOT.chat_stream('请帮我加入文心一言的元素后，重新写一首诗')

百度文心一言新，智能对话引风潮。 语言理解深似海，回答精准似琼瑶。 网络世界畅游行，知识问答如飞鸟。 科技魅力人人赞，文心一言领未来。


In [23]:
ERNIE_BOT.chat_stream('帮我解释一下前两句')

“百度文心一言新，智能对话引风潮”这两句诗主要描述了百度文心一言的新颖性和它在智能对话领域的引领作用。“百度文心一言新”主要强调了百度文心一言作为一种新的、先进的技术的出现；“智能对话引风潮”则描述了百度文心一言的智能对话功能引领了科技潮流，引发了广泛关注和讨论。


## Gradio应用体验
具体调用方式见[quick_start](../examples/quick_start)